# Introduction

UCI has a complex google doc they've been using to track their samples so far.

https://docs.google.com/spreadsheets/d/13M6-Ry6oXgkx94BHZOGioYPI6F_hWDqjGgcaNu2JNYs/edit#gid=1838362486

Also there's a metadata spreadsheet for IGVF at
https://docs.google.com/spreadsheets/d/1BLMledzmqOqXnJHzpijgw91IOs-9tSlVeZDG_MtXddk/edit#gid=1284120531

In [1]:
import bz2
from collections import Counter, namedtuple
import datetime
import numpy
import os
import pandas
from pathlib import Path
import re
from subprocess import run, PIPE
import sys
import zoneinfo

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mousedemo.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
from django.contrib.auth import get_user_model
from django.db import DEFAULT_DB_ALIAS

MOUSEDEMO = str(Path("mousedemo").absolute())
if MOUSEDEMO not in sys.path:
    sys.path.append(MOUSEDEMO)
    

# Pre-initialization setup

Backup the old ones and make a new database file before we initialize the database.

In [2]:
# Create a new database
result = run(["python3", "manage.py", "check"], capture_output=True, cwd=MOUSEDEMO)

if len(result.stderr) > 0:
    print(result.stderr.decode("utf-8"))
    
result.check_returncode()

In [3]:
initial = Path("igvf_mice/migrations/0001_initial.py")
if initial.exists():
    initial.unlink()
    
result = run(["python3", "manage.py", "makemigrations", "igvf_mice"], capture_output=True, cwd=MOUSEDEMO)

if len(result.stderr) > 0:
    print(result.stderr.decode("utf-8"))
    
result.check_returncode()

In [4]:
# Backup several old sqlite database files

db_name = Path("db.sqlite3")

backup_names = []
for i in range(0, 4):
    backup_names.append(Path("db{}.sqlite3".format(i+1)))

if backup_names[-1].exists():
    backup_names[-1].unlink()

if db_name.exists():    
    for i in reversed(range(0, 3)):
        if backup_names[i].exists():
            print("renaming {} to {}".format(backup_names[i], backup_names[i+1]))
            backup_names[i].rename(backup_names[i+1])

    db_name.rename(backup_names[0])
    print("renaming {} to {}".format(db_name, backup_names[0]))

renaming mousedemo/db3.sqlite3 to mousedemo/db4.sqlite3
renaming mousedemo/db2.sqlite3 to mousedemo/db3.sqlite3
renaming mousedemo/db1.sqlite3 to mousedemo/db2.sqlite3
renaming mousedemo/db.sqlite3 to mousedemo/db1.sqlite3


In [5]:
# Create a new database
result = run(["python3", "manage.py", "migrate"], capture_output=True, cwd=MOUSEDEMO)

if len(result.stderr) > 0:
    print(result.stderr.decode("utf-8"))
    
result.check_returncode()
    
assert Path("db.sqlite3").exists()

# Setup users

Now that we have a fresh clean database, lets create user accounts

In [6]:
django.setup()

from mousedemo import settings
from igvf_mice import models

In [7]:
# Create accounts.
with open("pw", "rt") as instream:
    pw = instream.read().strip()

user_model = get_user_model()
user_model._default_manager.db_manager(DEFAULT_DB_ALIAS).create_superuser(
    username="diane",
    password=pw,   
)
pw = None

# Load data

In [8]:
#spreadsheet_name = "IGVF_Split-seq_20230118.xlsx"
spreadsheet_name = "https://woldlab.caltech.edu/nextcloud/index.php/s/eEtjBfDqQFnLpSS/download"


In [9]:
book = pandas.ExcelFile(spreadsheet_name)
for name in book.sheet_names:
    print(name)

Planning
mice - 8 founders
Line information
schedule - all
Nuclei isolation Schedule with 
Counting
Samples - 8 founders
Bridge Samples
Samples into experiment
Experiment
Parvin practice Samples into ex
DCC
Plate setups
PBMCs
practice mice
IGVF002 Pilot Samples
IGVF002 Pulverized Samples
IGVF002 Plate Setup
IGVF002 Metadata
Cerebellum testing


Converters

In [10]:
def truncate(model):
    table_name = model._meta.db_table
    assert "\\" not in table_name
    with django.db.connection.cursor() as cursor:
        # I don't know why the sql params didn't work. this is a sql vulnerability waiting to happen
        cursor.execute("delete from \"{}\"".format(table_name))
        cursor.execute("DELETE FROM SQLITE_SEQUENCE WHERE name=\"{}\"".format(table_name))
        cursor.fetchone()

In [11]:
def int_or_none(x):
    if x == "N/A":
        return None
    elif pandas.isnull(x):
        return None
    else:
        return int(x)
    
def int_or_0(x):
    if pandas.isnull(x):
        return 0
    else:
        return int(x)

def float_or_none(x):
    if pandas.isnull(x):
        return None
    else:
        return float(x)
    
def str_or_empty(x):
    if pandas.isnull(x):
        return ""
    else:
        return x

def str_or_none(x):
    if pandas.isnull(x):
        return None
    else:
        return x
    
def date_or_none(x):
    if pandas.isnull(x):
        return None
    elif isinstance(x, datetime.datetime):
        return x.date()
    else:
        return x

def datetime_or_none(x):
    if pandas.isnull(x):
        return None
    else:
        return x


# initialize general use tables

In [12]:
source = [
    {"name": "The Jackson Laboratory", "url": "http://www.jax.org/index.html", "igvf_id": "/sources/jackson-labs"}
    {"name": "Parse Bioscience", "url": "https://www.parsebiosciences.com/", "igvf_id":},
    {"name": "Illumina", "url": "https://www.illumina.com", "igvf_id": "/sources/illumina/"},
]

if models.Source.objects.count() > 0:
    truncate(models.Source)

for row in source:
    record = models.Source(
        name=row["name"],
        url=row["url"])
    record.save()


# Library kit type & version

In [13]:
library_construction_kit = [
    {"name": "WT Mega", "version": "v2", "source": "Parse Bioscience"},
    #{"name": "", "version": "", "source": , "Illumina"},
]

if models.LibraryConstructionKit.objects.count() > 0:
    truncate(models.LibraryConstructionKit)

for row in library_construction_kit:
    try:
        source = models.Source.objects.get(name=row["source"])
        
        record = models.LibraryConstructionKit(
            name=row["name"],
            version=row["version"],
            source=source,
        )
        record.save()        
    except models.Source.DoesNotExist:
        print(f"Couldn't find {row['source']}")
        


# Load LibraryBarcodes

In [14]:
models.LibraryConstructionKit.objects.get(name="WT Mega")

<LibraryConstructionKit: WT Mega v2>

In [15]:
if models.LibraryBarcode.objects.count() > 0:
    truncate(models.LibraryBarcode)


# Load WT Mega v2 first barcode
bc1 = pandas.read_csv("bc_data_n192_v4.csv")

kit = models.LibraryConstructionKit.objects.get(name="WT Mega", version="v2")

for i, row in bc1.iterrows():
    record = models.LibraryBarcode(
        kit=kit,
        name=row["uid"],
        code=row["well"],
        sequence=row["sequence"],
        barcode_type=row["type"], # TODO what do the codes mean again T,R?
    )
    record.save()

sublibrary=[
    ("1", "CAGATC"),
    ("2", "ACTTGA"),
    ("3", "GATCAG"),
    ("4", "TAGCTT"),
    ("5", "ATGTCA"),
    ("6", "CTTGTA"),
    ("7", "AGTCAA"),
    ("8", "AGTTCC"),
    ("9", "GAGTGG"),
    ("10", "CCGTCC"),
    ("11", "GTAGAG"),
    ("12", "GTCCGC"),
    ("13", "GTGAAA"),
    ("14", "GTGGCC"),
    ("15", "GTTTCG"),
    ("16", "CGTACG"),
]
sublibrary = pandas.DataFrame(sublibrary, columns=["code","sequence"])

for i, row in sublibrary.iterrows():
    record = models.LibraryBarcode(
        kit=kit,
        code=row["code"],
        sequence=row["sequence"],
    )
    record.save()


# Initialize TissueOntology

In [16]:
tissue_column = pandas.read_excel(
    spreadsheet_name, 
    "Samples - 8 founders",
    usecols=["Tissue"])
tissue_column.head()

tissues = Counter()
for i, row in tissue_column.iterrows():
    if not pandas.isnull(row["Tissue"]):
        tissues[row["Tissue"]] += 1

tissues

Counter({'Hypothalamus/Pituitary': 81,
         'Cerebellum': 81,
         'Cortex/Hippocampus left': 81,
         'Cortex/Hippocampus right': 81,
         'Liver': 81,
         'Heart': 81,
         'Lung': 81,
         'Adrenal': 81,
         'Kidney': 81,
         'Gonads -1 Ovary': 41,
         'Gonads -2 Oviduct': 41,
         'Perigonadal fat': 81,
         'Brown fat': 81,
         'Soleus': 81,
         'Plantaris': 81,
         'Gastrocnemius': 81,
         'TA': 81,
         'EDL': 81,
         'Tail': 81,
         'PBMC - WBC': 81,
         'Gonads -1 Testis': 40,
         'Gonads -2 Epididymis': 40})

In [17]:
tissue_dissection_to_ontology_map = {
    'Hypothalamus/Pituitary': [("UBERON:0001898","hypothalamus"), ("UBERON:0000007","pituitary gland")],
    'Cerebellum': [("UBERON:0002037","cerebellum")],
    # Hippocampus might be:
    #   Hippocampal formation UBERON:0002421 https://www.ebi.ac.uk/ols/ontologies/uberon/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FUBERON_0002421
    #   Layer of hippocampus UBERON:0002305 https://www.ebi.ac.uk/ols/ontologies/uberon/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FUBERON_0002305
    'Cortex/Hippocampus left': [("NTR:0000646","left cerebral cortex"), ("NTR:0000750", "Hippocampal formation left")],
    'Cortex/Hippocampus right': [("NTR:0000647","right cerebral cortex"), ("NTR:0000751", "Hippocampal formation right")],
    'Liver': [("UBERON:0002107", "liver")],
    'Heart': [("UBERON:0000948","heart")],
    'Lung': [("UBERON:0002048","lung")],
    'Adrenal': [("UBERON:0002369","adrenal gland")],
    'Kidney': [("UBERON:0002113","kidney")],
    'Gonads -1 Ovary': [("UBERON:0000992","ovary")],
    'Gonads -2 Oviduct': [("UBERON:0000993","oviduct")],
    'Perigonadal fat': [("UBERON:0003428", "gonadal fat pad")], #"is closer" to dissection
    'Brown fat': [("UBERON:0001348","brown adipose tissue")], 
    'Soleus': [("UBERON:0001389","soleus muscle")],
    'Plantaris': [("UBERON:0011905","plantaris")],
    'Gastrocnemius': [("UBERON:0001388","gastrocnemius")],
    'TA': [("UBERON:0001385","tibialis anterior")],
    'EDL': [("UBERON:0001386","extensor digitorum longus")],
    'Tail': [("UBERON:0002415","tail")],
    'PBMC - WBC': [("CL:2000001", "peripheral blood mononuclear cell")],
    'Gonads -1 Testis': [("UBERON:0000473","testis")],
    'Gonads -2 Epididymis': [("UBERON:0001301","epididymis")],
}

term_details = pandas.read_csv("obo.tsv.bz2", compression="bz2", sep="\t", index_col="term_id")

if models.OntologyTerm.objects.count() > 0:
    truncate(models.OntologyTerm)
    
for key in tissue_dissection_to_ontology_map:
    for term_curie, term_name in tissue_dissection_to_ontology_map[key]:
        if term_curie.startswith("NTR:"):
            description=None
        else:
            details = term_details.loc[term_curie]
            description = details.description

        record = models.OntologyTerm(
            curie=term_curie,
            name=term_name,
            description=description
        )
        record.save()
    

In [18]:
strain_urls = {
    "AJ": "http://www.informatics.jax.org/inbred_strains/mouse/docs/A.shtml",
    "B6J": "http://www.informatics.jax.org/inbred_strains/mouse/docs/C57BL.shtml",
    "129S1J": "http://www.informatics.jax.org/inbred_strains/mouse/docs/129.shtml",
    "NODJ": "http://www.informatics.jax.org/inbred_strains/mouse/docs/NOD.shtml",
    "NZOJ": "http://www.informatics.jax.org/inbred_strains/mouse/docs/NZO.shtml",
    "CASTJ": "http://www.informatics.jax.org/inbred_strains/mouse/docs/CAST.shtml",
    "PWKJ": "http://www.informatics.jax.org/inbred_strains/mouse/docs/PWK.shtml",
    "WSBJ": "https://www.informatics.jax.org/strain/MGI:2160667"
}

strain_background = {
    "AJ": "A/J (AJ)",
    "B6J": "C57BL/6J (B6)",
    "129S1J": "129S1/SvImJ (129)",
    "NODJ": "NOD/ShiLtJ (NOD)",
    "NZOJ": "NZO/H1LtJ (NZO)",
    "CASTJ": "CAST/EiJ (CAST)",
    "PWKJ": "PWK/PhJ (PWK)",
    "WSBJ": "WSB/EiJ (WSB)",
    #"CAST (M. m. castaneus)",
    #"WSB (M. m. domesticus)",
    #"PWK (M. m. musculus)",
}

In [19]:
def int_csv_to_hex(x):
    r, g, b = x.split(",")
    
    return "#{:02x}{:02x}{:02x}".format(int(r), int(g), int(b))

def remove_citation(x):
    return x.split(" ")[0]

strains = pandas.read_excel(
    spreadsheet_name, 
    sheet_name="Line information", 
    usecols=[
        "Designation",
        "Strain",
        "Note",
        "Jax Catalog No",
        "Sample CODE",
        "Strain notes",
    ],
    converters={
        "Strain": remove_citation,
        "Jax Catalog No": str,
        "Strain notes": str_or_empty,
    }
)

# Remove the extra lines with the bridge samples for now. I don't want to track thta level of detail
bridge = strains[strains["Designation"] == "Bridge sample"].first_valid_index() - 1
strains = strains.iloc[0:bridge]

# Add in Strain URL
strains["url"] = strains["Sample CODE"].apply(lambda x: strain_urls[x])
strains

,Designation,Strain,Note,Jax Catalog No,Sample CODE,Strain notes,url
0,A,A/J,CC founder,000646,AJ,Yellower adrenal gland.,http://www.informatics.jax.org/inbred_strains/...
1,B,C57BL/6J,CC founder,000664,B6J,NaN,http://www.informatics.jax.org/inbred_strains/...
2,C,129S1/SvImJ,CC founder,002448,129S1J,Male skin is tougher than female skin. Gallbla...,http://www.informatics.jax.org/inbred_strains/...
3,D,NOD/ShiLtJ,CC founder,001976,NODJ,NaN,http://www.informatics.jax.org/inbred_strains/...
4,E,NZO/HlLtJ,CC founder,002105,NZOJ,NaN,http://www.informatics.jax.org/inbred_strains/...
5,F,CAST/EiJ,CC founder,000928,CASTJ,NaN,http://www.informatics.jax.org/inbred_strains/...
6,G,PWK/PhJ,CC founder,003715,PWKJ,NaN,http://www.informatics.jax.org/inbred_strains/...
7,H,WSB/EiJ,CC founder,001145,WSBJ,NaN,https://www.informatics.jax.org/strain/MGI:216...


In [20]:
strain_type_lookup = {
    "CC founder": models.StrainType.CC_FOUNDER,
    "CC Cross": models.StrainType.CC_CROSS,
}

if models.MouseStrain.objects.count() > 0:
    truncate(models.MouseStrain)

for i, row in strains.iterrows():
    record = models.MouseStrain(
        name=row["Strain"],
        code=strain_background[row["Sample CODE"]],
        strain_type=strain_type_lookup[row["Note"]],
        jax_catalog_number=row["Jax Catalog No"],
        notes=row["Strain notes"],
        url=row["url"]
    )
    record.save()

This is closest to the Mouse table

In [21]:
def estrus_cycle(x):
    if pandas.isnull(x):
        return "NA"
    else:
        return x

sex_lookup = {
    numpy.nan: models.SexEnum.UNKNOWN,
    "Female": models.SexEnum.FEMALE,
    "Male": models.SexEnum.MALE,
}

estrus_stage = {
    "Anestrus": models.EstrusCycle.ANESTRUS,
    "Anestrus>Proestrus": models.EstrusCycle.ANESTRUS_PROESTRUS,
    "Proestrus": models.EstrusCycle.PROESTRUS,
    "Proestrus>Estrus": models.EstrusCycle.PROESTRUS_ESTRUS,
    "Estrus": models.EstrusCycle.ESTRUS,
    "Estrus>Metestrus": models.EstrusCycle.ESTRUS_METESTRUS,
    "Metestrus": models.EstrusCycle.METESTRUS,
    "Metestrus>Diestrus": models.EstrusCycle.METESTRUS_DIESTRUS,
    "Diestrus": models.EstrusCycle.DIESTRUS,
    "Diestrus>Proestrus": models.EstrusCycle.DIESTRUS_PROESTRUS,
}
    
mice = pandas.read_excel(
    spreadsheet_name, 
    sheet_name="mice - 8 founders", 
    usecols=range(0,16),
    converters={
        "Estres cycle stage": estrus_cycle,
        "Dissection ID": int,
        "RMS number": int_or_none,
        "Housing number": int_or_none,
    }
)

if models.Mouse.objects.count() > 0:
    truncate(models.Mouse)

los_angeles_tz = zoneinfo.ZoneInfo("America/Los_Angeles")

failed = False
for i, row in mice.iterrows():
    if not pandas.isnull(row["Mouse Name"]):
        name = row["Mouse Name"]
        strain_code = row["Strain"]
        
        if name == "056_WSBJ_10F->PWKJ_9F":
            name = "056_PWKJ_9F"

        try:
            strain = models.MouseStrain.objects.get(name=strain_code)
        except models.MouseStrain.DoesNotExist as e:
            print(f"Unable to find {row['Strain']} for row {i+2}")
            failed = True
            continue

        if not name[-1] in ("M", "F"):
            raise ValueError(f"Unrecognized sex field {mouse_name}")
            
        name_fields = name.split("_")
        
        if len(name_fields) != 3:
            raise ValueError(f"Wrong number of attributes in {name} line {i+1}")
            
        if not name_fields[1] in strain_background:
            raise ValueError(f"strain background in {name} not recognized line {i+1}")
        
        sample_box = "" if pandas.isnull(row["Sample box"]) else row["Sample box"]
        
        record = models.Mouse(
            # should i use liz's disection id?
            name=name,
            strain=strain,
            sex=sex_lookup[row["Sex"]],
            weight_g=row["Weight (g)"],
            date_of_birth=row["DOB"].date(),
            operator=row["Operator"],
            notes=str_or_empty(row["Comments"]),
            sample_box=sample_box,
        )
        
        if not pandas.isnull(row["Estres cycle stage"]):
            # Ignore those transitions.
            stage = row["Estres cycle stage"]
            if stage.endswith("?"):
                stage = stage[:-1]
            record.estrus_cycle=estrus_stage[stage]
            
        if not pandas.isnull(row["Harvest start time"]):
            record.harvest_start_time=datetime.datetime(
                row["Harvest Date"].year,
                row["Harvest Date"].month,
                row["Harvest Date"].day,
                row["Harvest start time"].hour,
                row["Harvest start time"].minute,
                row["Harvest start time"].second,
                tzinfo=los_angeles_tz,
            )

        if not pandas.isnull(row["Harvest finish time"]):
            record.harvest_end_time=datetime.datetime(
                row["Harvest Date"].year,
                row["Harvest Date"].month,
                row["Harvest Date"].day,
                row["Harvest finish time"].hour,
                row["Harvest finish time"].minute,
                row["Harvest finish time"].second,
                tzinfo=los_angeles_tz,
            )
        record.save()
        
assert not failed, "Check warning messages"

# Tissues

In [22]:
#if models.Tissue.objects.count() > 0:
#    truncate(models.Tissue)

tissues = pandas.read_excel(
    spreadsheet_name, 
    sheet_name="Samples - 8 founders",
    header=0,
)
tissues

,IGVF Sample BOX,Mouse_Tissue ID,Unnamed: 2,Tissue,Sex,Timepoint,Genotype,Tube label,tube weight (g),tube+tissue wight (g),tissue weight (mg) or blood volume (ul),cryopreserved blood location,Dissection date,Approx. sac time,Mouse_Tissue ID.1,Dissector,Comment
0,IGVF_Sample_5&6,016_B6J_10F_01,NaN,Hypothalamus/Pituitary,F,10 weeks,B6J,016-01,1.046,1.118,72.0,NaN,2022-10-27,09:01:00,016_B6J_10F_01,SK,NaN
1,IGVF_Sample_5&6,016_B6J_10F_02,NaN,Cerebellum,F,10 weeks,B6J,016-02,1.046,NaN,-1046.0,NaN,2022-10-27,09:01:00,016_B6J_10F_02,SK,NaN
2,IGVF_Sample_5&6,016_B6J_10F_03,NaN,Cortex/Hippocampus left,F,10 weeks,B6J,016-03,1.040,1.181,141.0,NaN,2022-10-27,09:01:00,016_B6J_10F_03,SK,NaN
3,IGVF_Sample_5&6,016_B6J_10F_04,NaN,Cortex/Hippocampus right,F,10 weeks,B6J,016-04,1.048,NaN,-1048.0,NaN,2022-10-27,09:01:00,016_B6J_10F_04,SK,NaN
4,IGVF_Sample_5&6,016_B6J_10F_05,NaN,Liver,F,10 weeks,B6J,016-05,1.133,2.099,966.0,NaN,2022-10-27,09:01:00,016_B6J_10F_05,GM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [23]:
def parse_timepoint(value):
    value, units = value.split(" ")
    value = float(value)
    
    unit_map = {
        "day": models.AgeUnitsEnum.DAY,
        "days": models.AgeUnitsEnum.DAY,
        "week": models.AgeUnitsEnum.WEEK,
        "weeks": models.AgeUnitsEnum.WEEK,
        "month": models.AgeUnitsEnum.MONTH,
        "months": models.AgeUnitsEnum.MONTH,
    }
    return (value, unit_map[units])


In [24]:
if models.Tissue.objects.count() > 0:
    truncate(models.Tissue)

failed = False
for i, row in tissues.iterrows():
    if not pandas.isnull(row["Tissue"]):
        mouse_tissue_id_label = "Mouse_Tissue ID"
        mouse_tissue_name_fields = row[mouse_tissue_id_label].split("_")
        
        # some validation on tissue name
        if len(mouse_tissue_name_fields) != 4:
            raise ValueError(f"Not enough elements in mouse tissue id {row[mouse_tissue_id_label]}")
            
        if not mouse_tissue_name_fields[1] in strain_background:
            raise ValueError(f"tissue strain field in {row[mouse_tissue_id_label]} not recognized")
        
        mouse_name = "_".join(mouse_tissue_name_fields[0:3])
        try:
            mouse = models.Mouse.objects.get(name=mouse_name)
        except models.Mouse.DoesNotExist:
            print("row {}, {} was not found".format(i+2, mouse_name))
            failed = True
            continue

        genotype = strain_background[row["Genotype"]]
        
        # this is the "label swap" on spreadsheet rows 602-605.
        if mouse_name == "092_CASTJ_10F":
            genotype = "CAST/EiJ (CAST)"
        # this is the other half the swap on spreadsheet rows 1522-1525
        elif mouse_name == "046_NZOJ_10F":
            genotype = 'NZO/H1LtJ (NZO)'

        assert mouse.strain.code == genotype, f"{mouse.mouse_strain.code} != {genotype}"
        tissue_terms = []
        for term_curie, term_name in tissue_dissection_to_ontology_map[row["Tissue"]]:
            tissue_terms.append(models.OntologyTerm.objects.get(pk=term_curie))
        
        #age, age_units = parse_timepoint(row["Timepoint"])
        
        if pandas.isnull(row["Approx. sac time"]):
            sac_time = datetime.time(0,0,0)
        else:
            sac_time = datetime.time(
                row["Approx. sac time"].hour,
                row["Approx. sac time"].minute,
                row["Approx. sac time"].second,    
            )
        dissection = datetime.datetime(
            row["Dissection date"].year,
            row["Dissection date"].month,
            row["Dissection date"].day,
            sac_time.hour,
            sac_time.minute,
            sac_time.second,
            tzinfo=los_angeles_tz,
        )
        
        record = models.Tissue(
            mouse=mouse,
            name = row[mouse_tissue_id_label],
            dissection_time=dissection,
            #age=age,
            #age_units=age_units,
            tube_label=row["Tube label"],
            life_stage=models.LifeStageEnum.ADULT,
            dissector=row["Dissector"],
            dissection_notes=row["Comment"],
        )

        tube_weight_label = "tube weight (g)"
        if not pandas.isnull(row[tube_weight_label]):
            record.tube_weight_g = float(row[tube_weight_label])

        total_weight_label = "tube+tissue wight (g)"
        if not pandas.isnull(row[total_weight_label]):
            record.total_weight_g = float(row[total_weight_label])
        
        record.save()
        record.ontology_term.set(tissue_terms)
        record.save()

assert not failed, "Check warning messages."

In [25]:
strain_background

{'AJ': 'A/J (AJ)',
 'B6J': 'C57BL/6J (B6)',
 '129S1J': '129S1/SvImJ (129)',
 'NODJ': 'NOD/ShiLtJ (NOD)',
 'NZOJ': 'NZO/H1LtJ (NZO)',
 'CASTJ': 'CAST/EiJ (CAST)',
 'PWKJ': 'PWK/PhJ (PWK)',
 'WSBJ': 'WSB/EiJ (WSB)'}

# Information from Samples into experiment

- Total barcoded nuclei (Samples into experiment)

Is this FixedTissue, FixedSample, FixedBiosample?



In [26]:
samples_into_experiment = pandas.read_excel(
    spreadsheet_name, 
    sheet_name="Samples into experiment",
    header=[0,1],
    index_col=None,
)

level0 = []
level1 = []
for header in samples_into_experiment.columns:
    level0.append("" if header[0].startswith("Unnamed: ") else header[0])
    level1.append(header[1])

samples_into_experiment.columns = pandas.MultiIndex.from_arrays([level0, level1], names=["phase", "name"])

if models.FixedSample.objects.count() > 0:
    truncate(models.FixedSample)

one_based = 1
header_lines = 2
failed = False
for i, row in samples_into_experiment.iterrows():
    line_no = i + one_based + header_lines
    box_name = row[("", "IGVF Fixation BOX")]
    if isinstance(box_name, str):
        box_name = box_name.strip()
    
    if box_name in ["IGVF_FIX_001", "IGVF_FIX_002"]:
        continue

    tissue_id = row[("", "Mouse_Tissue ID")]
    pooled_from = row[("after fixation", "pooled_from")]
    
    if pandas.isnull(tissue_id):
        continue
    
    if (tissue_id.endswith("_25") or tissue_id.endswith("_26")) and pandas.isnull(pooled_from):
        print(f"How are we going to merge these samples? {tissue_id}")
        continue
        
    if pandas.isnull(pooled_from):
        pooled_from = [tissue_id]
    else:
        pooled_from = pooled_from.split(",")
    
    tissues = []
    for pooled_id in pooled_from:
        try:
            tissues.append(models.Tissue.objects.get(name=pooled_id))
        except models.Tissue.DoesNotExist:
            print(f"Tissue {pooled_id} not found in tissue table on {line_no}")
            failed = True

    weight = row[("", "weight (mg)")]
    fixation_date = row[("", "Fixation date")]
    notes = row[("", "Notes")]
            
    if weight == "#VALUE!":
        pass
    elif weight < 0:
        pass
    elif pandas.isnull(weight):
        pass
    else:
        for tissue in tissues:
            if not pandas.isnull(tissue.weight_mg) and tissue.weight_mg != weight:
                print(f"Tissue {pooled_from} weight does not match {weight} {tissue.weight_mg}. {line_no}")

    sample_id = row[("after fixation", "Sample ID")]
    if not (pandas.isnull(sample_id) or pandas.isnull(tissue_id)):
        if tissue_id != sample_id:
            print(f"If defined, {tissue_id} should equal {sample_id} line {line_no}")
            failed = True

    cap_label = row[("after fixation", "Cap label")]
    if not (pandas.isnull(cap_label) or pandas.isnull(sample_id)):
        sample_fields = sample_id.split("_")
        predicted_label = "_".join([sample_fields[0], sample_fields[-1]])
        if predicted_label != cap_label:
            print(f"{cap_label} should equal {predicted_label} line {line_no}")
            failed = True
    
    well_id = row[("Loading", "wells in Barcoding plate")]
        
    record = models.FixedSample(
        name=tissue_id,
        tube_label=cap_label,
        fixation_name=box_name,
        fixation_date=date_or_none(row[("", "Fixation date")]),
        starting_nuclei=int_or_none(row[("before fixation", "Total nuclei (x10^6)")]),
        nuclei_into_fixation=int_or_none(row[("before fixation", "Nuclei into fixation (x10^6)")]),
        fixed_nuclei=int_or_none(row[("after fixation", "Total fixed nuclei (x10^6)")]),
        aliquots_made=int_or_none(row[("after fixation", "# aliquots")]),
        aliquot_volume_ul=float_or_none(row[("after fixation", "uL per aliquot")]),
    )
    record.save()
    record.tissue.set(tissues)
    record.save()

assert not failed, "Check warning messages"


Tissue ['046_NZOJ_10F_03'] weight does not match 115.0 150.0. 157
Tissue ['046_NZOJ_10F_01'] weight does not match 42.0 76.0. 457
Tissue ['046_NZOJ_10F_17'] weight does not match 190.0 116.0. 556
Tissue ['016_B6J_10F_10', '016_B6J_10F_11'] weight does not match 33.0 17.0. 600
Tissue ['016_B6J_10F_10', '016_B6J_10F_11'] weight does not match 33.0 16.0. 600
Tissue ['017_B6J_10M_10', '017_B6J_10M_11'] weight does not match 263.0 180.0. 601
Tissue ['017_B6J_10M_10', '017_B6J_10M_11'] weight does not match 263.0 83.0. 601
Tissue ['018_B6J_10F_10', '018_B6J_10F_11'] weight does not match 42.0 25.0. 602
Tissue ['018_B6J_10F_10', '018_B6J_10F_11'] weight does not match 42.0 17.0. 602
Tissue ['019_B6J_10M_10', '019_B6J_10M_11'] weight does not match 280.0 178.0. 603
Tissue ['019_B6J_10M_10', '019_B6J_10M_11'] weight does not match 280.0 102.0. 603
Tissue ['020_B6J_10F_10', '020_B6J_10F_11'] weight does not match 40.0 20.0. 604
Tissue ['020_B6J_10F_10', '020_B6J_10F_11'] weight does not match 40

# Plate layout

(the hard thing)

In [27]:
WellContent = namedtuple("well_content", ["genotype", "tissue_id"])

def validate_tissue_ids(contents, expected_genotypes):
    for expected, (row_index, row) in zip(expected_genotypes, contents.iterrows()):
        for cell in row:
            fields = cell.split("_")
            assert len(fields) == 4, f"Field label validation fail {cell}"
            assert fields[1] == expected, f"{fields[1]} failed to match {expected}"

def find_plate_start(sheet, offsets):
    for plate_id_row in sheet[sheet[offsets["plate_label"]].apply(lambda x: not pandas.isnull(x) and x.startswith("IGVF_"))].index:
        plate_name = sheet.loc[plate_id_row, offsets["plate_label"]]
        for i in range(plate_id_row, plate_id_row + 4):
            cell = sheet.loc[i, offsets["well_start"]]
            if isinstance(cell, str) and cell.startswith("Tissue"):
                yield (plate_name, i)
                
def get_plate_genotype(sheet, start, offsets):
    plate_tissues = sheet.loc[start+2:start+9, offsets["tissue_label"]].tolist()
    
    return plate_tissues

def parse_single_well_block(sheet, plate_start, offsets):
    # column ids
    simple_plate_column_ids = sheet.loc[plate_start+1, offsets["well_range"]].tolist()

    plate_row_labels = sheet.loc[plate_start+2:plate_start+9, offsets["well_row_label"]].tolist()

    contents = sheet.loc[plate_start+2:plate_start+9, offsets["well_range"]].copy()
    contents.index = plate_row_labels
    contents.columns = simple_plate_column_ids

    genotypes = get_plate_genotype(sheet, plate_start, offsets)
    validate_tissue_ids(contents, genotypes)

    well_contents = {}
    for genotype, (well_row, row) in zip(genotypes, contents.iterrows()):
        for well_column, cell in zip(simple_plate_column_ids, row):
            well_contents[(well_row, str(well_column))] = [WellContent(genotype, cell)]
    return well_contents

def parse_multiplexed_well_block(sheet, start, expected_genotypes, offsets):
    well_re = re.compile("^[A-H]1?[\d]$")
    wells = sheet.loc[start+3, offsets["well_range"]].tolist()
    for cell in wells:
        if pandas.isnull(cell) or well_re.match(cell) is None:
            raise ValueError(f"well value {cell} in row {start+3} does not look correct")

    contents = sheet.loc[start+0:start+1, offsets["well_range"]]
    contents.columns = [(x[0], x[1:]) for x in wells]
    validate_tissue_ids(contents, expected_genotypes)
    contents.index = expected_genotypes

    well_contents = {}
    for genotype, row in contents.iterrows():
        for well, tissue_id in row.iteritems():
            well_contents.setdefault(well, []).append(WellContent(genotype, tissue_id))

    return well_contents

def parse_plate(sheet):
    max_rows_from_plate_name_to_well_names = 4

    offsets = {
        "plate_label": 1,
        "tissue_label": 1,
        "well_row_label": 2,
        "well_start": 3,
        "well_range": slice(3, 10)
    }

    for plate_name, plate_start in find_plate_start(sheet, offsets):
        well_contents = {}
        genotypes = get_plate_genotype(sheet, plate_start, offsets)
        well_contents.update(parse_single_well_block(sheet, plate_start, offsets))
        
        complex_blocks = [
            (plate_start + 12, slice(0, 2)),
            (plate_start + 12 + 5, slice(2, 4)),
            (plate_start + 12 + 10, slice(4, 6)),
            (plate_start + 12 + 15, slice(6, 8)),
        ]
        
        for complex_start, genotype_slice in complex_blocks:
            expected_genotypes = genotypes[genotype_slice]
            well_contents.update(parse_multiplexed_well_block(sheet, complex_start, expected_genotypes, offsets))
        
        yield (plate_name, well_contents)
        
plate_layout = pandas.read_excel(
    spreadsheet_name, 
    sheet_name="Plate setups",
    header=None,
)


# Validate tissue references:
failed = False
for plate_name, plate_contents in parse_plate(plate_layout):
    for well_id in plate_contents:
        well_contents = plate_contents[well_id]
        
        for well_fraction in well_contents:
            try:
                biosample = models.Tissue.objects.get(name=well_fraction.tissue_id)
            except models.Tissue.DoesNotExist:
                print(f"Unable to find {well_fraction.tissue_id} on plate {plate_name}")
                failed = True
                
assert not failed, "Resolve tissues"

# populate database
if models.SplitSeqWell.biosample.through.objects.count() > 0:
    truncate(models.SplitSeqWell.biosample.through)

if models.SplitSeqWell.objects.count() > 0:
    truncate(models.SplitSeqWell)
    
if models.SplitSeqPlate.objects.count() > 0:
    truncate(models.SplitSeqPlate)

wt_mega_2_kit = models.LibraryConstructionKit.objects.get(name="WT Mega", version="v2")

for plate_name, plate_contents in parse_plate(plate_layout):
    plate_record = models.SplitSeqPlate(
        name=plate_name,
        size=models.PlateSizeEnum.size_96,
        pool_location=None,
        date_performed=None,
    )
    plate_record.save()
    
    for well_id in plate_contents:
        well_contents = plate_contents[well_id]
        
        biosamples = []
        for well_content in well_contents:
            try:
                biosamples.append(models.FixedSample.objects.get(name=well_content.tissue_id))
            except models.Tissue.DoesNotExist:
                print(f"unable to find tissue {well_content.tissue_id} for {plate_name} {well_id}")
        
        barcodes = models.LibraryBarcode.objects.filter(
            kit=wt_mega_2_kit,
            code="{}{}".format(well_id[0], well_id[1]),
        )
        
        well_record = models.SplitSeqWell(
            plate=plate_record,
            row=well_id[0],
            column=well_id[1],
        )
        well_record.save()
        well_record.biosample.set(biosamples)
        well_record.barcode.set(barcodes)
        well_record.save()


# Sublibraries

## Extract meaning from the Experiments tab

- number of nuclei per sublibrary (Experiment) 
- cDNA amp # PCR cycles (Experiment)
- cDNA ng/ul in 25ul (experiment)
- total cDNA ng (experiment)
- Bioanalyzer cDNA ave bp length (expierment)
- Sub library Index PCR # (experiment)
- Sublibrary Index (experiment)
- Sequence for SampleSheet (experiment)
- library ng/ul (experiment)
- Bioanalyzer library ave bp length (experiment)
- Nextseq run number (experiment)

- number of 67k aliquots  (experiment)
- number of 8k aliquots (experiment)

- QC # raw reads (1 mismatch) (experiment)

In [28]:
experiment = pandas.read_excel(
    spreadsheet_name, 
    sheet_name="Experiment",
    header=0,
    index_col=None,
    usecols=range(0, 49)
).rename(columns={
    "Unnamed: 40": "Novaseq raw reads",
    "Unnamed: 41": "Novaseq L001_1",
    "Unnamed: 42": "Novaseq L002_1",
    "Unnamed: 43": "Novaseq L003_1",
    "Unnamed: 44": "Novaseq L004_1",
    "Unnamed: 45": "Novaseq L001_2",
    "Unnamed: 46": "Novaseq L002_2",
    "Unnamed: 47": "Novaseq L003_2",
    "Unnamed: 48": "Novaseq L004_2",
})

experiment

,Experiment,mice,Sample Box,Fixation Box,Split-seq prep start date,barcoded [nuclei/ul] cell counter,Volume of barcoded nuclei,Total barcoded nuclei (1st 3 rounds),# of backup 67k aliquots,# of backup 8k aliquots,...,Unnamed: 39,Novaseq raw reads,Novaseq L001_1,Novaseq L002_1,Novaseq L003_1,Novaseq L004_1,Novaseq L001_2,Novaseq L002_2,Novaseq L003_2,Novaseq L004_2
0,IGVF_Splitseq_002,B6 (2F+2M),IGVF_Sample_3,IGVF_FIX_001,2022-08-24 00:00:00,4540,240.0,1089600.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,CAST (2F+2M),IGVF_Sample_4,IGVF_FIX_002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fixation V1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1M V1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
def find_fixation_start(sheet):
    for fixation_id_row in sheet[sheet["Experiment"].apply(lambda x: not pandas.isnull(x) and x.startswith("IGVF_Splitseq"))].index:
        yield (sheet.loc[fixation_id_row, "Experiment"],fixation_id_row)

list(find_fixation_start(experiment))

[('IGVF_Splitseq_002', 0),
 ('IGVF_Splitseq_003', 17),
 ('IGVF_Splitseq_004', 35),
 ('IGVF_Splitseq_005', 53),
 ('IGVF_Splitseq_006', 71),
 ('IGVF_Splitseq_007', 90),
 ('IGVF_Splitseq_008', 108),
 ('IGVF_Splitseq_009', 127)]

In [30]:
def parse_fixation(sheet, experiment_name, start):
    block = sheet.loc[start: start+15]

    experiment_name_fields = experiment_name.split("_")
    plate_name = f"{experiment_name_fields[0]}_{experiment_name_fields[-1]}"
    
    experiment = {
        "experiment_name": experiment_name,
        "plate_name": plate_name,
        "prep_date": block.loc[start, "Split-seq prep start date"],
        "aliquots_8k": int_or_0(block.loc[start, "# of backup 8k aliquots"]),
        "aliquots_67k": int_or_0(block.loc[start, "# of backup 67k aliquots"]),
        "sublibrary": [],
    }

    for i, row in block.iterrows():
        if pandas.isnull(row["NovaSeq Sequencing date"]):
            novaseq_run_id = None
        else:
            novaseq_run_id = row["NovaSeq Sequencing date"].isoformat().split("T")[0]

        if pandas.isnull(row["Novaseq raw reads"]):
            novaseq_raw_reads = None
        elif row["Novaseq raw reads"] == "Total":
            novaseq_raw_reads = None
        else:
            novaseq_raw_reads = int(row["Novaseq raw reads"])
        
        sublibrary = {
            "name": row["sublibrary"],
            "nuclei": int_or_none(row["# of nuclei per sublibrary"]),
            "cdna_pcr_rounds": row["cDNA amp # PCR cycles"],
            "cdna_ng_per_ul_in_25ul": row["[cDNA] ng/ul     in 25ul"],
            #total_cdna = cdna_ng_per_ul_in_25ul * 25
            "bioanalyzer_date": row["BA date"],
            "cdna_average_bp_length": int_or_none(row["Bioanalyzer cDNA ave bp length"]),
            "index_pcr_no": int_or_none(row["Sub library Index PCR #"]),
            "index": str(int_or_none(row["Sublibrary Index"])),
            "barcode": row["Sequence for SampleSheet"],
            "library_ng_per_ul": float_or_none(row["[library] ng/ul"]),
            "library_average_bp_length": float_or_none(row["Bioanalyzer library ave bp length"]),
            "nextseq_run_date": date_or_none(row["QC Sequencing date"]),
            "nextseq_run_id": str_or_none(int_or_none(row["Nextseq run #"])),
            "nextseq_run_raw_reads": int_or_none(row["QC # raw reads  (1 mismatch)"]),
            "novaseq_run_date": date_or_none(row["NovaSeq Sequencing date"]),
            "novaseq_run_id": novaseq_run_id,
            "novaseq_run_raw_reads": novaseq_raw_reads,
        }
        experiment["sublibrary"].append(sublibrary)
        if pandas.isnull(sublibrary["nuclei"]):
            continue
        elif sublibrary["nuclei"] < 10000:
            experiment["aliquots_8k"] += 1
        elif sublibrary["nuclei"] > 60000 and sublibrary["nuclei"] < 70000:
            experiment["aliquots_67k"] += 1
        else:
            print(f"Unusual number of nuclei {sublibrary['name']} {sublibrary['nuclei']}")
    return experiment

parse_fixation(experiment, "IGVF_Splitseq_003", 17)

{'experiment_name': 'IGVF_Splitseq_003',
 'plate_name': 'IGVF_003',
 'prep_date': datetime.datetime(2022, 12, 1, 0, 0),
 'aliquots_8k': 3,
 'aliquots_67k': 22,
 'sublibrary': [{'name': '003_8A',
   'nuclei': 8000,
   'cdna_pcr_rounds': '5 + 10',
   'cdna_ng_per_ul_in_25ul': 130.0,
   'bioanalyzer_date': Timestamp('2022-12-05 00:00:00'),
   'cdna_average_bp_length': 1150,
   'index_pcr_no': 12,
   'index': '1',
   'barcode': 'CAGATC',
   'library_ng_per_ul': 11.8,
   'library_average_bp_length': 443.0,
   'nextseq_run_date': datetime.date(2022, 12, 6),
   'nextseq_run_id': 1049,
   'nextseq_run_raw_reads': 112237518,
   'novaseq_run_date': None,
   'novaseq_run_id': None,
   'novaseq_run_raw_reads': None},
  {'name': '003_67A',
   'nuclei': 67000,
   'cdna_pcr_rounds': '5 + 7',
   'cdna_ng_per_ul_in_25ul': 239.0,
   'bioanalyzer_date': Timestamp('2022-12-05 00:00:00'),
   'cdna_average_bp_length': 1149,
   'index_pcr_no': 12,
   'index': '2',
   'barcode': 'ACTTGA',
   'library_ng_per_u

In [31]:
failed = False
for experiment_name, experiment_start in find_fixation_start(experiment):
    fixation = parse_fixation(experiment, experiment_name, experiment_start)
    
    try:
        plate = models.SplitSeqPlate.objects.get(name=fixation["plate_name"])
    except models.SplitSeqPlate.DoesNotExist:
        print(f"Unable to find plate {fixation['plate_name']}")
        continue
        
    plate.aliquots_small_made = fixation["aliquots_8k"]
    plate.aliquots_large_made = fixation["aliquots_67k"]
    plate.save()
    
    wt_mega_2_kit = models.LibraryConstructionKit.objects.get(name="WT Mega", version="v2")
    for sublibrary in fixation["sublibrary"]:
        try:
            barcodes = models.LibraryBarcode.objects.filter(kit=wt_mega_2_kit, barcode_type=None, code=sublibrary["index"])
        except models.LibraryBarcode.DoesNotExist:
            print(f"Unable to find barcode {sublibrary['index']}")

        # validate barcodes
        library_barcodes_sequence = {b.sequence for b in barcodes}
        expected_sequence = set(sublibrary["barcode"].split(","))
        if library_barcodes_sequence != expected_sequence:
            print(f"Database lookup of barcodes {library_barcodes_sequence} doesn't match human entry {expected_sequence}")
            failed = True
            continue
        
        record = models.Sublibrary(
            name=sublibrary["name"],
            plate=plate,
            nuclei=sublibrary["nuclei"],
            cdna_pcr_rounds=sublibrary["cdna_pcr_rounds"],
            cdna_ng_per_ul_in_25ul=float_or_none(sublibrary["cdna_ng_per_ul_in_25ul"]),
            bioanalyzer_date=date_or_none(sublibrary["bioanalyzer_date"]),
            cdna_average_bp_length=int_or_none(sublibrary["cdna_average_bp_length"]),
            index_pcr_number=int_or_none(sublibrary["index_pcr_no"]),
            index=str(int_or_none(sublibrary["index"])),
            library_ng_per_ul=float_or_none(sublibrary["library_ng_per_ul"]),
            library_average_bp_length=int_or_none(sublibrary["library_average_bp_length"]),
        )
        record.save()
        record.barcode.set(barcodes)
        record.save()
        
        if sublibrary["nextseq_run_date"] is not None:
            run = models.SequencingRun(
                platform="NE",
                name=sublibrary["nextseq_run_id"],
                library=record,
                run_date=sublibrary["nextseq_run_date"],
                raw_reads=sublibrary["nextseq_run_raw_reads"],
            )
            run.save()
            
        if sublibrary["novaseq_run_date"] is not None:
            run = models.SequencingRun(
                platform="NO",
                name=sublibrary["novaseq_run_id"],
                library=record,
                run_date=sublibrary["novaseq_run_date"],
                raw_reads=sublibrary["novaseq_run_raw_reads"],
            )
            run.save()
      
assert not failed

Unable to find plate IGVF_002
Unable to find plate IGVF_007
Unable to find plate IGVF_008
Unable to find plate IGVF_009
